In [1]:
import QuantLib as ql
import pandas as pd

today = ql.Date(7, ql.January, 2013)
ql.Settings.instance().evaluationDate = today

In [2]:
dates, forwards = zip(*[(ql.Date(7, 1, 2013), 0.03613672438543303),
                        (ql.Date(8, 4, 2013), 0.03613672438543303),
                        (ql.Date(8, 7, 2013), 0.033849133719219514),
                        (ql.Date(7, 1, 2014), 0.03573931373272106),
                        (ql.Date(7, 7, 2014), 0.03445303757052511)])
libor_curve = ql.ForwardCurve(dates, forwards, ql.Actual365Fixed())

In [3]:
index = ql.GBPLibor(ql.Period(6, ql.Months), ql.YieldTermStructureHandle(libor_curve))
calendar = index.fixingCalendar()
nominal = 1000000
length = 1
maturity = calendar.advance(today, length, ql.Years)
adjustment = index.businessDayConvention()
schedule = ql.Schedule(today, maturity, index.tenor(), calendar, adjustment, adjustment, ql.DateGeneration.Backward, False)
floating_leg = ql.IborLeg([nominal], schedule, index, index.dayCounter())

In [4]:
df = pd.DataFrame()

dates = list(schedule)
df['fixing date'] = dates[:-1]
df['index fixing'] = [index.fixing(d) for d in df['fixing date']]
df['start date'] = dates[:-1]
df['end date'] = dates[1:]
df['days'] = df['end date'] - df['start date']
df['accrual period'] = df['days'] / 365
df['amount'] = df['index fixing'] * nominal * df['accrual period']

df

,fixing date,index fixing,start date,end date,days,accrual period,amount
0,"January 7th, 2013",0.035300,"January 7th, 2013","July 8th, 2013",182,0.49863,17601.6
1,"July 8th, 2013",0.036056,"July 8th, 2013","January 7th, 2014",183,0.50137,18077.4


In [5]:
df2 = pd.DataFrame({'amount': [c.amount() for c in floating_leg], 'rate': [ql.as_coupon(c).rate() for c in floating_leg]})

df2

,amount,rate
0,17601.643836,0.035300
1,18080.116395,0.036061


In [6]:
coupon = ql.as_floating_rate_coupon(floating_leg[1])
print(coupon.fixingDate())
print(index.fixing(coupon.fixingDate()))

July 8th, 2013
0.036056087457623655


In [7]:
startDate = index.valueDate(coupon.fixingDate())
endDate = index.maturityDate(startDate)
print(startDate)
print(endDate)

July 8th, 2013
January 8th, 2014


In [8]:
print(libor_curve.forwardRate(startDate, endDate, coupon.dayCounter(), ql.Simple))

3.605609 % Actual/365 (Fixed) simple compounding


In [9]:
couponStart = coupon.accrualStartDate()
couponEnd = coupon.accrualEndDate()
print(couponStart)
print(couponEnd)

July 8th, 2013
January 7th, 2014


In [10]:
print(libor_curve.forwardRate(couponStart, couponEnd, coupon.dayCounter(), ql.Simple))

3.606143 % Actual/365 (Fixed) simple compounding


In [11]:
coupon.rate()

0.0360614343399347

In [12]:
coupon.rate() * nominal * coupon.accrualPeriod()

18080.116395090554

In [13]:
coupon.amount()

18080.11639509055